In [2]:
import requests, datetime, time, json
from bs4 import BeautifulSoup

def httpReq(board, indexNum):
    index_url = 'https://www.ptt.cc/bbs/' + board + '/index' + str(indexNum) + '.html'
    index_res = requests.get(index_url)
    return BeautifulSoup(index_res.text, 'html5lib')

def link_list(board, indexNum):
    link_list = []
    for link in httpReq(board, indexNum).select('.r-ent'):
        link_list.append('https://www.ptt.cc' + link.select('a')[0]['href'])
    return link_list

def grap_push_info(soup):
    pushInfo_list = []
    for i in soup.select('.push'):
        if len(i.select('.push-ipdatetime')[0].string.strip()) > 11:
            continue
        pushInfo_list.append(
                              # date (e.g. '02/18')
                             [i.select('.push-ipdatetime')[0].string.strip(), 
                              # userid (e.g. 'comdan66')
                              i.select('.push-userid')[0].string.strip(), 
                              # push content
                              i.select('.push-content')[0].string[1:].strip()]
        )
    return pushInfo_list

def grap_orig_post(soup):
    return (
        # date (e.g. 'Mon Aug 29 14:32:25 2005')
        soup.select('.article-meta-value')[3].text.strip() +'``!@'+
        # title
        soup.select('.article-meta-value')[2].text.strip() +'``!@'+
        # userid (e.g. 'skyboy')
        soup.select('.article-meta-value')[0].text.split('(')[0].strip() +'``!@'+                   
        # content
        soup.select('#main-content')[0].contents[4].strip()
    ).split('``!@')

def get_year(soup, push_m, push_d):
    m_dict = {'jan':1, 'feb':2, 'mar':3, 'apr':4, 'may':5, 'jun':6, 
              'jul':7, 'aug':8, 'sep':9, 'oct':10, 'nov':11, 'dec':12}
    
    post_y = int(soup.select('.article-meta-value')[3].text.strip()[-4:])
    post_m = m_dict[soup.select('.article-meta-value')[3].text.strip()[4:7].lower()]
    
    if push_m < post_m:
        return str(post_y+1)
    else:
        return str(post_y)

def crawl_in_page(board, indexNum):
    for link in link_list(board, indexNum):
        try:
            con_res = requests.get(link)
            con_soup = BeautifulSoup(con_res.text, 'html5lib')
        except:
            counter = 1
            while counter <= 3:
                print('Retry after 2 sec...')
                time.sleep(2)
                con_res = requests.get(link)
                con_soup = BeautifulSoup(con_res.text, 'html5lib')
                counter += 1
        try:
            # crawl main content
            ptt_data.append({
                'date'   : grap_orig_post(con_soup)[0],
                'title'  : grap_orig_post(con_soup)[1],
                'userid' : grap_orig_post(con_soup)[2],
                'content': grap_orig_post(con_soup)[3],
                'url'    : con_res.url,
                'is_orig_post': True
            })

            # crawl push content
            for i in grap_push_info(con_soup):

                ptt_data.append({
                    'date'   : get_year(con_soup, int(i[0][0:2]), int(i[0][3:5])) + '/' + i[0],
                    'title'  : grap_orig_post(con_soup)[1],
                    'userid' : i[1],
                    'content': i[2],
                    'url'    : con_res.url,
                    'is_orig_post': False
                })

            with open('ptt_data.json', 'w') as outfile:
                json.dump(ptt_data, outfile)
            
        except:
            continue
            

ptt_data = []
board = ['Yunlin', 'Lifeismoney']
indexCounter = 1
boardCounter = 0
crawl_in_page(board[boardCounter], indexCounter)

#while True:
#    try:
#        crawl_in_page(board[boardCounter], indexCounter)
#    except:
#        crawl_in_page(board[boardCounter+1], indexCounter)
#        break



In [52]:
m_dict = {'jan':1, 'feb':2, 'mar':3, 'apr':4, 'may':5, 'jun':6, 
              'jul':7, 'aug':8, 'sep':9, 'oct':10, 'nov':11, 'dec':12}

m_dict['jan'] = 1
m_dict.items()

dict_items([('dec', 12), ('jan', 1), ('sep', 9), ('nov', 11), ('jun', 6), ('apr', 4), ('feb', 2), ('mar', 3), ('may', 5), ('aug', 8), ('jul', 7), ('oct', 10)])